In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

In [2]:
from ICARUS.Database import DB
from ICARUS.Database.Database_3D import Database_3D
from time import sleep
import os

No forces.lspt file found in hermes_3 folder at /home/tryfonas/data/Uni/Software/hermes/Data/3D!


# Open Database

In [3]:
db3d: Database_3D = DB.vehicles_db

### Select Plane

In [4]:
planenames = db3d.get_planenames()
for i, planename in enumerate(planenames):
    print(f"{i} : `{planename}") 
sleep(1)
indexes = [1]# range(0,len(planenames))
#index = int(input("Select a plane: "))
plane = db3d.planes[planenames[1]]
# dynplane = db3d.dyn_planes[planenames[index]]
planenames = [planenames[i] for i in indexes]

0 : `bmark
1 : `hermes_3


# Add XFLR Polars

In [5]:
from ICARUS.Input_Output.XFLR5.polars import read_polars_3d
from ICARUS.Database import XFLRDB
for name in planenames:
    if name.startswith("XFLR"):
        continue
    if name.endswith("_dyn"):
        name = name[:-4]
    if name.endswith("_hd"):
        name = name[:-3]
    if name.endswith("_3") or name.endswith("_7"):
        name = name[:-2]
        
    if f"XFLR_{name}" not in planenames: 
        try:
            XFLR5PLANEDIR: str = os.path.join(XFLRDB,f"{name}.txt")
            read_polars_3d(XFLR5PLANEDIR, name)
            print(f"Imported XFLR polar for {name}")
            planenames.append(f"XFLR_{name}")
        except FileNotFoundError:
            pass
            # print(f"No XFLR polar found for {name}")
        

Imported XFLR polar for hermes


In [6]:
# planenames =[planenames[0]]
# planenames[0] = "Embraer_E190_Cruise 2D"
# db3d.data[planenames[0]] = db3d.data["XFLR_e190_cruise"]

In [7]:
# try:
#     XFLR5PLANEDIR = os.path.join(XFLRDB,f"e190_takeoff.txt")
#     read_polars_3d(db3d, XFLR5PLANEDIR, 'XFLR5_e190_takeoff')
#     planenames.append('Embraer_E190_takeoff')
#     planenames[-1]= "Embraer_E190_takeoff"
# except FileNotFoundError:
#     print("No XFLR polar found for e190_takeoff.txt")

In [8]:
# db3d.data[planenames[1]] = db3d.data["XFLR_Embraer_E190_takeoff"]

# Airplane Visualization

In [9]:
# plane.visualize()

# Polars

In [10]:
db3d.data['bmark']['GNVP3 Potential CL']#.keys()

0   -0.000783
1   -0.000523
2   -0.000261
3         NaN
4    0.000261
5    0.000523
6    0.000783
Name: GNVP3 Potential CL, dtype: float64

In [11]:
from ICARUS.Visualization.airplane.db_polars import plot_airplane_polars 
solvers = [
            "GNVP3 Potential",
            "GNVP3 2D", 
            # "GNVP7 Potential",
            # "GNVP7 2D",
            # "LSPT Potential",
            # "LSPT 2D"
        ]
axs, fig = plot_airplane_polars(
    planenames,
    solvers,
    # plots = [["AoA", "CL"], ["AoA", "CD"]],
    size=(6,7),
)

In [ ]:
from ICARUS.Visualization.airplane.cg_investigation import cg_investigation

cg_investigation(
    planenames[0],
    solvers,
    size=(10,7),
)

In [ ]:
err

# Case Convergence

In [ ]:
from ICARUS.Visualization.airplane.gnvp_convergence import plot_convergence

In [14]:
angles2show = [1.,2.]# All
plot_convergence(
    plane.name,
    angles = angles2show,
    solvers= ["2D"],
    plot_error = False,size=(16,7)
)

NameError: name 'plot_convergence' is not defined

# Wake Visualization

In [ ]:
from ICARUS.Visualization.airplane.gnvp_wake import plot_gnvp_wake
from ICARUS.Database.utils  import angle_to_case

In [ ]:
CASE: str = angle_to_case(-3.)
plane = db3d.planes[plane.name]
# plane.visualize()
plot_gnvp_wake(3,plane,CASE)

 # Loads Visualization

In [ ]:
from ICARUS.Visualization.airplane.gnvp_strips import gnvp_strips_2d, gnvp_strips_3d
NBs = [i+1 for i in range(len(plane.surfaces))]
print(plane)
print("NBs to Visualize: ",NBs)

stripData = gnvp_strips_2d(plane,CASE, NBs[1], category= 'Wind')

# Dynamics

In [ ]:
states = db3d.states['hermes_3']
for i,name in enumerate(states):
    print(f"{i}: {name}")

In [ ]:
# name = 'atlas'
from ICARUS.Flight_Dynamics.state import State


state: State = states[name]
state.stability_fd()
state.eigenvalue_analysis()
print(state)
state.plot_eigenvalues()
#   Longitudinal derivatives
#   Xu=    -0.16379         Cxu=   -0.049477
#   Xw=      1.1494         Cxa=     0.34723
#   Zu=     -3.0117         Czu=  -0.0035415
#   Zw=     -19.528         CLa=      5.8991
#   Zq=     -1.8321         CLq=      9.1645
#   Mu= -0.00060463         Cmu=  -0.0015122
#   Mw=    -0.45664         Cma=     -1.1421
#   Mq=    -0.39098         Cmq=     -16.193

#   Lateral derivatives
#   Yv=     -0.5496         CYb=    -0.16602
#   Yp=   -0.017022         CYp=  -0.0045504
#   Yr=     0.32383         CYr=    0.086568
#   Lv=    0.039731         Clb=   0.0053106
#   Lp=     -5.6227         Clp=     -0.6651
#   Lr=      1.0377         Clr=     0.12275
#   Nv=     0.28879         Cnb=    0.038602
#   Np=    -0.57586         Cnp=   -0.068117
#   Nr=     -0.1259         Cnr=   -0.014893


In [ ]:
state.SBderivativesDS.tree()

# Sensitivity Plot

In [ ]:
from ICARUS.Visualization.airplane.gnvp_sensitivity import plot_sensitivity 

In [ ]:
# pertr = dynplane.pertubResults
# print(dynplane.trim)
# trim = pertr[pertr['Type'] == 'Trim']
# dynplane.sensResults['u'].sort_values(by=['Epsilon'])
# plotSensitivity(dynplane.sensResults,dynplane,trim,relative = 0,vars2s=['r'])